In [1]:
# PyGame template.
 
# Import standard modules.
import numpy as np
import random
from collections import deque
from tensorflow.keras.layers import Dense,Input,Multiply
from tensorflow.keras.models import Model
import tensorflow as tf


actions = [(-1,0),(1,0),(-1,1),(1,1)]
batch_size = 64
state_shape = 13
action_size = 4
learning_rate =  0.00025
hidden_neurons = 512
random_decay = 0.01
memory_gamma = 0.9
WIZARD_HEIGHT = 103

def step(bullets,enemy_spawner,wizard,action,state):
    done = False
    reward = 0
    if (wizard.y - vel > 0) and (action[0]==1) :
        wizard.y -= action[0]*vel
    if (wizard.y + WIZARD_HEIGHT +vel<height) and (action[0]==-1):
        wizard.y -= action[0]*vel
    state[0]=wizard.y
    for enemy in enemy_spawner.enemy_group:
        if wizard.colliderect(enemy):
          reward -= 10
          done = True
          break  
    if(not done):
        reward += 1
    for bullet in bullets:    
        for enemy in enemy_spawner.enemy_group:
          if bullet.colliderect(enemy):
            reward +=10
    if (action[1]==1) and len(bullets)<MAX_BULLET :
        bullet= pygame.Rect(wizard.x + wizard.width , wizard.y + wizard.height//2 , 10 , 5)
        bullets.append(bullet)
    return state,reward,done    

class Memory():

    def __init__(self, memory_size, batch_size):
        self.buffer = deque (maxlen=memory_size)

        self.batch_size = batch_size

    def add(self, experience):

# Adds a (state, action, reward, state_prime, done) tuple. 
        self.buffer.append(experience)

    def sample(self):

        buffer_size = len(self.buffer)

        index= np.random.choice(np.arange(buffer_size), size=self.batch_size)

        
        batch = [self.buffer[i] for i in index]

        return batch

memory = Memory(memory_size= 1000000 ,batch_size=64  )

def deep_q_network (state_shape, action_size, learning_rate, hidden_neurons):
    state_input = Input((state_shape,), name='frames') 
    actions_input = Input((action_size,), name='mask')

    hidden_1 = Dense(hidden_neurons, activation=tf.keras.activations.relu)(state_input)

    hidden_2 = Dense(hidden_neurons, activation=tf.keras.activations.relu) (hidden_1)
    q_values = Dense(action_size)(hidden_2)

    masked_q_values = Multiply()([q_values, actions_input])

    model= Model(inputs = [state_input, actions_input], outputs=masked_q_values)
    optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

def act(state, training=False):
    random_rate = 1
    random_decay = 0.01
    if training:

# Random actions until enough simulations to train the model.

        if len(memory.buffer) >= memory.batch_size:
            random_rate= random_decay

        if random_rate > np.random.rand(): return random.randint(0, action_size-1)

# If not acting randomly, take action with highest predicted value.

    state_batch = np.expand_dims (state, axis=0)
    predict_mask = np.ones((1, action_size,))
    action_qs = network.predict([state_batch, predict_mask])
    return np.argmax (action_qs[0])

network = deep_q_network (state_shape, action_size, learning_rate, hidden_neurons)

def update_Q():
    sample = memory.sample()
    state_mb= sample[0] 
    action_mb = sample[1]
    reward_mb = sample[2]
    state_prime_mb = sample[3]
    done_mb = sample[4]

# Get Q values for state_prime_mb.
    print(state_prime_mb)
    predict_mask = np.ones((1, action_size,))
    next_q_mb= network.predict([state_prime_mb[0], predict_mask])
    next_q_mb= tf.math.reduce_max (next_q_mb, axis=1)

# Apply the Bellman Equation

    target_qs = (next_q_mb * memory.gamma) + reward_mb
    target_qs = tf.where(done_mb, reward_mb, target_qs)

# Match training batch to network output

    action_mb = tf.convert_to_tensor (action_mb, dtype=tf.int32)
    action_hot = tf.one_hot (action_mb, action_size)
    target_mask = tf.multiply(tf.expand_dims (target_qs, -1), action_hot)
    return network.train_on_batch([state_mb, action_hot], target_mask)

import sys
import os
from matplotlib.pyplot import show
 
# Import non-standard modules.
import pygame
from pygame.locals import *
from button import Button
from enemy_spawner import EnemySpawner
from random import randrange
import time


pygame.init() 
pygame.display.set_caption("Welcome Day") #The window title

#Import images and define sizes
width, height = 720, 480 
WIZARD_WIDTH , WIZARD_HEIGHT = 78 , 103
LIFE_WIDTH , LIFE_HEIGHT = 21 , 22 
CARA_WIZARD_IMAGE =pygame.image.load(os.path.join('Assets','cara.png')) #character image

CARA_WIZARD=pygame.transform.scale(CARA_WIZARD_IMAGE,(WIZARD_WIDTH,WIZARD_HEIGHT)) #resize

BULLET_FIRE=pygame.image.load(os.path.join('Assets','fireball.png'))
BULLET_FIRE=pygame.transform.scale(BULLET_FIRE, (28,12))
screen = pygame.display.set_mode((width, height))
BULLET_VEL= 7
vel=5
width, height = 720, 480 
MAX_BULLET = 3
bullets=[]
menu=pygame.image.load(os.path.join('Assets','menu.png'))
menu=pygame.transform.scale(menu,(width,height))
background=pygame.image.load(os.path.join('Assets','sky.jpg')) #background image
background=pygame.transform.scale(background, (width,height))
gameover=pygame.image.load(os.path.join('Assets','gameover.jpg'))
gameover=pygame.transform.scale(gameover, (width, height))
pygame.font.init()
pygame.mixer.init()
fireball_sfx = pygame.mixer.Sound(os.path.join('SFX','fireball.mp3'))
font = pygame.font.Font(os.path.join('Fonts', 'FreeSansBold.ttf'), 32)
textX = 10
textY = 10
score = 0
lounge = pygame.mixer.music.load(os.path.join('SFX','lounge.mp3'))

def show_score(x,y, screen):
  
  score_text = font.render("Score: " + str(score), True, (255, 255, 255))
  screen.blit(score_text,(x,y))
  pygame.display.flip()



def update(wizard):
  # Go through events that are passed to the script by the window.
   for event in pygame.event.get():
        if event.type == pygame.QUIT:
          pygame.quit()
          sys.exit()
        if event.type ==pygame.KEYDOWN:
          if event.key == pygame.K_SPACE  and len(bullets)<MAX_BULLET : # event to shot bullets
            bullet= pygame.Rect(wizard.x + wizard.width , wizard.y + wizard.height//2 , 10 , 5)
            bullets.append(bullet)
 
def draw(screen , wizard,bullets, enemy_spawner):
 
  screen.fill((0, 0, 0)) # Fill the screen with black.
  screen.blit(background,(0,0))
  # Redraw screen here.
  screen.blit(CARA_WIZARD , (wizard.x , wizard.y)) # display character

  enemy_spawner.enemy_group.draw(screen)
  for bullet in bullets :
    screen.blit(BULLET_FIRE, bullet ) #display bullets 
  
  # Flip the display so that the things we drew actually show up.
  pygame.display.flip()

# function to handle the character mouvements (Up,Down)
def handle_movement(keys_pressed, wizard):
  if keys_pressed[pygame.K_UP] and wizard.y - vel > 0: #UP
    wizard.y-=vel
  if keys_pressed[pygame.K_DOWN] and wizard.y + vel + wizard.height < height : #DOWN
    wizard.y+=vel

#function to handle the bullet mouvement
def handle_bullet(bullets, wizard, enemy_spawner):
  
  for bullet in bullets:
    bullet.x += BULLET_VEL
    if wizard.colliderect(bullet):
      

      bullets.remove(bullet)
    elif bullet.x> width :
      bullets.remove(bullet)

    global score 
    for enemy in enemy_spawner.enemy_group:
      if bullet.colliderect(enemy):
        enemy.kill()
        score += 1
        pygame.mixer.Sound.play(fireball_sfx)
        bullets.remove(bullet)

def get_font(size): 
  return pygame.font.Font("Fonts/font.ttf", size)



def play():
  # Initialise PyGame.
  global score

  # Set up the clock. This will tick every frame and thus maintain a relatively constant framerate. Hopefully.
  fps = 60.0
  
  fpsClock = pygame.time.Clock()
  
  # Set up the window.

  
  enemy_spawner = EnemySpawner()

  #character
  wizard=pygame.Rect(20 ,100, WIZARD_WIDTH , WIZARD_HEIGHT )
  
  
  # Main game loop.
  dt = 1/fps # dt is the time since last frame.
  run=True
  game_over = False
  while run: # Loop forever!

    fpsClock.tick(fps) # You can update/draw here, I've just moved the code for neatness.
    
    if game_over == False:
      draw(screen , wizard , bullets, enemy_spawner)
      
      
      enemy_spawner.update()
      
      #handeling some events
      update(wizard)
        
      for enemy in enemy_spawner.enemy_group:
        if wizard.colliderect(enemy):
          
          game_over = True
     
      keys_pressed = pygame.key.get_pressed()
      show_score(560,40, screen)
      state = [wizard.y]
      for enemy in enemy_spawner.enemy_group:
        if enemy.rect.x>0:
            state.append(enemy.rect.x)
            state.append(enemy.rect.y)
      length = len(state)  
      for i in range(length,7):
        state.append(-1)
      for bullet in bullets:
        state.append(bullet.x)
        state.append(bullet.y)
      length = len(state)
      for i in range(length,13):
        state.append(-1)
      print(state)
      action= act(state,training = True)
      action = actions[action]  
      n_state,reward,done = step(bullets,enemy_spawner,wizard,action,state)
      exp = [state,action,reward,n_state,done]
      memory.add(exp)
      update_Q()
      handle_bullet(bullets,wizard, enemy_spawner)
    else:
      screen.blit(gameover,(0,0))
      pygame.display.update()
      for event in pygame.event.get():
        if event.type == pygame.QUIT:
          pygame.quit()
          sys.exit()
        if event.type == pygame.KEYDOWN:
          if event.key == pygame.K_RETURN:
            score=0
            play()
      
    

def main():
  
  fps = 60.0
  fpsClock = pygame.time.Clock()
  dt = 1/fps 
  pygame.mixer.music.set_volume(0.1)
  pygame.mixer.music.play()
  
  
  while True :
    fpsClock.tick(fps)
    screen.blit(menu,(0,0))

    PLAY_MOUSE_POS=pygame.mouse.get_pos() # get mouse position
    PLAY_BUTTON = Button(None, pos=(width/2 -3, height/2 +51), 
                            text_input="START", font=get_font(33), base_color="#ffdd00", hovering_color="#E6A2C7")
    PLAY_BUTTON.changeColor(PLAY_MOUSE_POS)
    PLAY_BUTTON.update(screen)
    for event in pygame.event.get():
      if event.type == pygame.QUIT:
        pygame.quit() # Opposite of pygame.init
        sys.exit() 
      if event.type == pygame.MOUSEBUTTONDOWN:
        if PLAY_BUTTON.checkForInput(PLAY_MOUSE_POS):
          for i in range(50000):  
              play()
    pygame.display.update()
 


main()

C:\Users\pc\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
[100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[[95, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], (1, 1), 1, [95, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], False]


C:\Users\pc\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).